In [3]:
# Kiểm tra torch và torchvision
import torch
import torchvision

print("Phiên bản torch:", torch.__version__)
print("Phiên bản torchvision:",torchvision.__version__)
print("Pytorch đã nhận gpu:", torch.cuda.is_available())

Phiên bản torch: 2.5.1+cu118
Phiên bản torchvision: 0.20.1+cu118
Pytorch đã nhận gpu: True


In [2]:
# Sử dụng thư viện ultralytics và kiểm tra phiên bản
import ultralytics
ultralytics.checks()

Ultralytics 8.3.86  Python-3.9.12 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Setup complete  (8 CPUs, 15.8 GB RAM, 231.1/631.5 GB disk)


In [3]:
# nếu bị trùng thư viện này thì dùng 
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [3]:
import os
import random
import shutil

# thiết lập đường dẫn thư mục chứa dữ liệu gốc và đầu ra
input_folder = r"E:\WorkspaceHDD\Labelima\data"

# thiết lập đường dẫn cho ảnh và nhãn của từng tập dữ liệu (train, val, test)
output_folder = r".\data"
images_train_folder = os.path.join(output_folder, "images/train")
images_val_folder = os.path.join(output_folder, "images/val")
images_test_folder = os.path.join(output_folder, "images/test")
labels_train_folder = os.path.join(output_folder, "labels/train")
labels_val_folder = os.path.join(output_folder, "labels/val")
labels_test_folder = os.path.join(output_folder, "labels/test")

# tạo thư mục nếu chưa tồn tại để tránh lỗi khi lưu dữ liệu
os.makedirs(images_train_folder, exist_ok=True)
os.makedirs(images_val_folder, exist_ok=True)
os.makedirs(images_test_folder, exist_ok=True)
os.makedirs(labels_train_folder, exist_ok=True)
os.makedirs(labels_val_folder, exist_ok=True)
os.makedirs(labels_test_folder, exist_ok=True)

# lấy danh sách tất cả file .jpg trong thư mục gốc
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]

# shuffle dữ liệu để đảm bảo tính ngẫu nhiên
random.seed(1337)
random.shuffle(image_files)

# chia dữ liệu: 80% train, 10% val, 10% test
total_images = len(image_files)
train_split = int(0.8 * total_images)
val_split = int(0.9 * total_images)  # 80% train + 10% val

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

# hàm di chuyển file và nhãn tương ứng
def move_files(file_list, source_folder, dest_image_folder, dest_label_folder):
    for file_name in file_list:
        # Di chuyển file ảnh
        src_image = os.path.join(source_folder, file_name)
        dest_image = os.path.join(dest_image_folder, file_name)
        shutil.copy(src_image, dest_image)

        # Di chuyển file nhãn tương ứng
        label_name = file_name.replace('.jpg', '.txt')
        src_label = os.path.join(source_folder, label_name)
        dest_label = os.path.join(dest_label_folder, label_name)
        if os.path.exists(src_label):
            shutil.copy(src_label, dest_label)

# di chuyển file vào train, val, test
move_files(train_files, input_folder, images_train_folder, labels_train_folder)
move_files(val_files, input_folder, images_val_folder, labels_val_folder)
move_files(test_files, input_folder, images_test_folder, labels_test_folder)

print("Dữ liệu đã được chia thành công!")



Dữ liệu đã được chia thành công!


In [6]:
# đếm số lượng ảnh trong từng thư mục
train_count = len(os.listdir(images_train_folder))
val_count = len(os.listdir(images_val_folder))
test_count = len(os.listdir(images_test_folder))

print(f"Tổng số lượng ảnh: {total_images}")
print(f"Số lượng ảnh trong tập train: {train_count}")
print(f"Số lượng ảnh trong tập val: {val_count}")
print(f"Số lượng ảnh trong tập test: {test_count}")

Tổng số lượng ảnh: 2767
Số lượng ảnh trong tập train: 2213
Số lượng ảnh trong tập val: 277
Số lượng ảnh trong tập test: 277


Huấn luyện mô hình

In [6]:
from ultralytics import YOLO

# load a pretrained model (recommended for training)
model = YOLO("yolo11n.pt")

# huấn luyện mô hình
model.train(
    data="./data/data.yaml",  # đường dẫn file yaml cho data
    epochs=60, # số epoch (số lần quét toàn bộ data để train)
    imgsz=512,  # kích thước ảnh đầu vào
    batch=4,   # số ảnh xử lý cùng 1 lúc
    name="yolo11n-nhandienbienso",  # thư mục lưu mô hình sau khi train
    workers=4,   # số luồng xử lý khi load data
    device=0,  # sử dụng GPU để huấn luyện
    amp=False  # tắt amp do gtx 1650 hỗ trợ không tốt fp16, tránh mất thông tin
)

# @software{yolo11_ultralytics,
#   author = {Glenn Jocher and Jing Qiu},
#   title = {Ultralytics YOLO11},
#   version = {11.0.0},
#   year = {2024},
#   url = {https://github.com/ultralytics/ultralytics},
#   orcid = {0000-0001-5950-6979, 0000-0002-7603-6750, 0000-0003-3783-7069},
#   license = {AGPL-3.0}
# }

Ultralytics 8.3.86  Python-3.9.12 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=./data/data.yaml, epochs=60, time=None, patience=100, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=yolo11n-nhandienbienso, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning E:\WorkspaceHDD\DoAn\VIETNAMESE-LICENSE-PLATE\YOLO\data\labels\train.cache... 2213 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2213/2213 [00:00<?, ?it/s]
val: Scanning E:\WorkspaceHDD\DoAn\VIETNAMESE-LICENSE-PLATE\YOLO\data\labels\val.cache... 277 images, 0 backgrounds, 0 corrupt: 100%|██████████| 277/277 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolo11n-nhandienbienso\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 512 train, 512 val
Using 4 dataloader workers
Logging results to runs\detect\yolo11n-nhandienbienso
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/60     0.773G     0.7104      1.245     0.9417          2        512: 100%|██████████| 554/554 [01:36<00:00,  5.75it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:03<00:00,  9.01it/s]


                   all        277        278      0.695      0.838      0.736      0.601

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        277        278          1          1      0.995      0.919



60 epochs completed in 1.360 hours.
Optimizer stripped from runs\detect\yolo11n-nhandienbienso\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\yolo11n-nhandienbienso\weights\best.pt, 5.5MB

Validating runs\detect\yolo11n-nhandienbienso\weights\best.pt...
Ultralytics 8.3.86  Python-3.9.12 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:03<00:00, 10.55it/s]


                   all        277        278          1          1      0.995       0.92
Speed: 0.3ms preprocess, 4.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\yolo11n-nhandienbienso


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000023FFD8C95B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

Nhận dạng vùng biển số

In [4]:
import cv2
from ultralytics import YOLO
# Load mô hình
model = YOLO("./runs/detect/yolo11n-nhandienbienso/weights/best.pt")

# Nhận diện và lấy kết quả
results = model("test.JPG")

# Lấy ảnh đã có bounding box
for result in results:
    image_and_boxes = result.plot()

    for box in result.boxes.xyxy:  # Duyệt qua các bounding box
        x1, y1, x2, y2 = map(int, box)  # Lấy tọa độ từng box
        plates = image_and_boxes[y1:y2, x1:x2]  # Cắt vùng biển số từ ảnh gốc

# Hiển thị ảnh
cv2.imshow("Detected", image_and_boxes)
cv2.waitKey(0)
cv2.destroyAllWindows()



image 1/1 E:\WorkspaceHDD\DoAn\VIETNAMESE-LICENSE-PLATE\Training\test.JPG: 512x384 1 biensoxe, 54.8ms
Speed: 5.2ms preprocess, 54.8ms inference, 231.9ms postprocess per image at shape (1, 3, 512, 384)


So sánh mô hình

In [1]:
from ultralytics import YOLO

# Load model
model = YOLO("./runs/detect/yolo11n-nhandienbienso/weights/best.pt")

# Evaluate on test set
metrics = model.val(data="data/data.yaml", split="test")


Ultralytics 8.3.86  Python-3.9.12 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning E:\WorkspaceHDD\DoAn\VIETNAMESE-LICENSE-PLATE\Training-yolo\data\labels\test.cache... 277 images, 0 backgrounds, 0 corrupt: 100%|██████████| 277/277 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:04<00:00,  3.79it/s]


                   all        277        278          1      0.999      0.995      0.934
Speed: 0.5ms preprocess, 5.3ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs\detect\val3


In [2]:
# Trích xuất các chỉ số
precision = metrics.box.mp       # mean precision
recall = metrics.box.mr         # mean recall
map_50 = metrics.box.map50      # mAP@0.5
f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)  # F1-score
inference_time = metrics.speed['inference']  # milliseconds
fps = 1000 / inference_time

print(f"\n===== Kết quả =====")
print(f"mAP@0.5       : {map_50:.4f}")
print(f"Precision     : {precision:.4f}")
print(f"Recall        : {recall:.4f}")
print(f"F1-Score      : {f1_score:.4f}")
print(f"FPS (Inference): {inference_time:.2f} ms => {fps:.2f} FPS")
print("=============================\n")


===== Kết quả =====
mAP@0.5       : 0.9950
Precision     : 1.0000
Recall        : 0.9994
F1-Score      : 0.9997
FPS (Inference): 5.30 ms => 188.75 FPS

